In [766]:
import modern_robotics as mr
import numpy as np
import sympy as sp
from sympy import*
from sympy.physics.mechanics import dynamicsymbols, mechanics_printing
mechanics_printing()

In [767]:
#Funksjoner som trengs

def PsFromTsd(T_sd):
    #Finner Ps fra T_sd
    #T_sd gir konfigurasjonen vi vil ha end-effector framen, B, i.
    #B, og derav også M, er lik som i DH
    #s er plassert nederst på roboten med positiv z oppover, altså ikke som i DH. Bør kanskje endres til å være lik DH 
    P_d = np.array([0,0,80,1])
    P_s = T_sd@P_d

    return P_s

def exp3(omega, theta):
    omega = skew(omega)
    R = sp.eye(3) + sp.sin(theta) * omega + (1 - sp.cos(theta)) * omega * omega
    return R

def skew(v):
    return Matrix([[0, -v[2], v[1]],
                    [v[2], 0, -v[0]],
                    [-v[1], v[0], 0]])
                    
def exp6(twist, theta):
    omega = skew(twist[:3])
    v = Matrix(twist[3:])
    T = eye(4)
    T[:3,:3] = exp3(twist[:3], theta)
    T[:3,3] = (eye(3) * theta + (1 - cos(theta)) * omega +
              (theta-sin(theta)) * omega * omega) * v
    return T


def rotX(alfa_im1):
    Rx = sp.eye(4)
    Rx[1,1] =    sp.cos(alfa_im1)
    Rx[1,2] =   -sp.sin(alfa_im1)
    Rx[2,1] =    sp.sin(alfa_im1)
    Rx[2,2] =    sp.cos(alfa_im1)
    return Rx

def rotZ(alfa_im1):
    Rz = sp.eye(4)
    Rz[0,0] =    sp.cos(alfa_im1)
    Rz[0,1] =   -sp.sin(alfa_im1)
    Rz[1,0] =    sp.sin(alfa_im1)
    Rz[1,1] =    sp.cos(alfa_im1)
    return Rz

def rotY(alfa_im1):
    Ry = sp.eye(4)
    Ry[0,0] =    sp.cos(alfa_im1)
    Ry[0,2] =    sp.sin(alfa_im1)
    Ry[2,0] =   -sp.sin(alfa_im1)
    Ry[2,2] =    sp.cos(alfa_im1)
    return Ry




In [1287]:
#Definerer S og M

S1 = np.array([0,0,-1,0,0,0])
S2 = np.array([0,1,0,-400,0,25])
S3 = np.array([0,1,0,-855,0,25])
S4 = np.array([-1,0,0,0,-890,0])
S5 = np.array([0,1,0,-890,0,445])
S6 = np.array([-1,0,0,0,-890,0])
Slist = np.array([S1,S2,S3,S4,S5,S6]).T
print(Slist)

M = np.array([[0,0,-1,525],
              [0,1,0,0],
              [1,0,0,890],
              [0,0,0,1]])

thetas_up = [0,0,0,0,0,0]
thetas_down = [0,0,0,0,0,0]


#Limits til roboten slik den er gitt i oppgaven. Antar at ledd 5 har limit på +-90
theta_limits = [[-180,180],[-190+90,45+90],[-120-90, 156-90],[-180,180],[-90,90],[-180,180]]

[[   0    0    0   -1    0   -1]
 [   0    1    1    0    1    0]
 [  -1    0    0    0    0    0]
 [   0 -400 -855    0 -890    0]
 [   0    0    0 -890    0 -890]
 [   0   25   25    0  445    0]]


In [1292]:

#OPPGAVE 3.2

#Her endres thetas_gen for å teste forskjellige konfigurasjoner:
thetas_gen = np.array([2,1,2,np.pi/2,np.pi/2,np.pi])
T_sd = mr.FKinSpace(M,Slist,thetas_gen)
print("T_sd\n", T_sd)

P_s = PsFromTsd(T_sd)
print("Ps", P_s)

#theta1
thetas_up[0] = -atan2(P_s[1],P_s[0]) #minus foran fordi vinkelen er definert andre vei
thetas_down[0] = thetas_up[0]

#theta2 and theta3
a = np.sqrt(420**2+35**2)
c = 455
P_merket = np.array([P_s[0],P_s[1],P_s[2]-400])
b = np.sqrt((np.sqrt(P_merket[0]**2+P_merket[1]**2)-25)**2 + P_merket[2]**2)#Merk: -25 pga offset i skulder

psi = np.arccos(420/a) #Vinkelen mellom den faktiske armen og den vi tegna for å få en trekant(Pga 35mm offset i elbow). Se notatbok
phi = atan2(P_merket[2], sqrt(P_merket[0]**2 + P_merket[1]**2)-25)

alpha = np.arccos((b**2+c**2-a**2)/(2*b*c))
beta = np.arccos((a**2+c**2-b**2)/(2*a*c))

thetas_up[1] = np.pi/2 - (alpha + phi) #Index 1 på thetas[1] og thetas[2] gir elbow down løsning
thetas_down[1] =  np.pi/2 - (phi-alpha)

thetas_up[2] = np.pi/2 - (beta-psi)
thetas_down[2] =  -(2*np.pi - (beta+psi) - np.pi/2)

print(thetas_up, thetas_down)

#Vi har XYX euler angles. De er egentlig (-X)Y(-X) fordi det er slik S'ene er definert,.

#Elbow down:
T1 = exp6(S1, -thetas_down[0])
T2 = exp6(S2, -thetas_down[1])
T3 = exp6(S3, -thetas_down[2])

R = (T3@T2@T1@T_sd@np.linalg.inv(M)) #R er den resterende rotasjonen vi ønsker å få fra de tre siste leddene, definert i s

thetas_down[3] = -atan2(R[1,0], -R[2,0]) #minus foran theta4 og 6 fordi de er i minus x retning
thetas_down[4] = atan2(sqrt(1-R[0,0]**2), R[0,0])
thetas_down[5] = -atan2(R[0,1], R[0,2])


#Elbow up:
T1 = exp6(S1, -thetas_up[0])
T2 = exp6(S2, -thetas_up[1])
T3 = exp6(S3, -thetas_up[2])

R = (T3@T2@T1@T_sd@np.linalg.inv(M))

thetas_up[3] = -atan2(R[1,0], -R[2,0])
thetas_up[4] = atan2(sqrt(1-R[0,0]**2), R[0,0])
thetas_up[5] = -atan2(R[0,1], R[0,2])

T_sd
 [[ -0.412   -0.0587   0.9093 -71.5002]
 [ -0.9002  -0.1283  -0.4161  36.009 ]
 [  0.1411  -0.99    -0.     551.9174]
 [  0.       0.       0.       1.    ]]
Ps [  1.2436   2.7173 551.9174   1.    ]
[-1.14159265358983, 1.81578717467677 - pi, 1.3101796208137904, 1.57079632679490, 1.57079632679490, -0.0863250895303041 + pi] [-1.14159265358983, 4.17961642332856 - pi, -4.285489810626702, 1.57079632679490, 1.57079632679490, 2.22044604925031e-15 - pi]


In [1293]:
#testing av analytisk løsning:

#UP
thetas_up_num = np.zeros(6)
thetas_up_deg = np.zeros(6)
for i in range(0,6):
    thetas_up_num[i] = N(thetas_up[i])
    thetas_up_deg[i] = np.rad2deg(thetas_up_num[i])

#print(thetas_deg, np.rad2deg(thetas_gen))

T_up_thetas = mr.FKinSpace(M,Slist,thetas_up_num)
P_up_reached = PsFromTsd(T_up_thetas)
#print(P_reached, P_s)
np.set_printoptions(precision=4)

#DOWN
thetas_down_num = np.zeros(6)
thetas_down_deg = np.zeros(6)
for i in range(0,6):
    thetas_down_num[i] = N(thetas_down[i])
    thetas_down_deg[i] = np.rad2deg(thetas_down_num[i])

#print(thetas_deg, np.rad2deg(thetas_gen))

T_down_thetas = mr.FKinSpace(M,Slist,thetas_down_num)
P_down_reached = PsFromTsd(T_down_thetas)


#fk_test = exp6(S4,thetas_num[3])@exp6(S5,thetas_num[4])@exp6(S6,thetas_num[5])@M
#R_test = rotX(thetas_num[3])@rotY(thetas_num[4])@rotX(thetas_num[5]) 
thetas_calc, asd = mr.IKinSpace(Slist,M,T_sd,[0,0,0,0,0,0],0.01,0.01)

T_sd, thetas_gen, T_up_thetas, thetas_up_num, T_down_thetas, thetas_down_num, P_s, P_up_reached, P_down_reached

(array([[ -0.412 ,  -0.0587,   0.9093, -71.5002],
        [ -0.9002,  -0.1283,  -0.4161,  36.009 ],
        [  0.1411,  -0.99  ,  -0.    , 551.9174],
        [  0.    ,   0.    ,   0.    ,   1.    ]]),
 array([2.    , 1.    , 2.    , 1.5708, 1.5708, 3.1416]),
 array([[ -0.412 ,  -0.0587,   0.9093, -71.5002],
        [ -0.9002,  -0.1283,  -0.4161,  36.009 ],
        [  0.1411,  -0.99  ,   0.    , 551.9174],
        [  0.    ,   0.    ,   0.    ,   1.    ]]),
 array([-1.1416, -1.3258,  1.3102, -1.5708,  1.5708,  2.9844]),
 array([[ -0.412 ,  -0.0587,   0.9093, -71.5002],
        [ -0.9002,  -0.1283,  -0.4161,  36.009 ],
        [  0.1411,  -0.99  ,   0.    , 551.9174],
        [  0.    ,   0.    ,   0.    ,   1.    ]]),
 array([-1.1416,  1.038 , -4.2855, -1.5708,  1.5708, -0.2475]),
 array([  1.2436,   2.7173, 551.9174,   1.    ]),
 array([  1.2436,   2.7173, 551.9174,   1.    ]),
 array([  1.2436,   2.7173, 551.9174,   1.    ]))

In [1295]:
T_sd, thetas_up_num, thetas_down_num


(array([[ -0.412 ,  -0.0587,   0.9093, -71.5002],
        [ -0.9002,  -0.1283,  -0.4161,  36.009 ],
        [  0.1411,  -0.99  ,  -0.    , 551.9174],
        [  0.    ,   0.    ,   0.    ,   1.    ]]),
 array([-1.1416, -1.3258,  1.3102, -1.5708,  1.5708,  2.9844]),
 array([-1.1416,  1.038 , -4.2855, -1.5708,  1.5708, -0.2475]))